In [1]:
# === AUTO GITHUB SYNC (Single-cell version) ===
import os, time, subprocess
from datetime import datetime

# === CONFIGURATION ===
WATCH_DIR = r"C:\Users\lokes\Documents\Projects\statistics-notebooks"  # your repo path
BRANCH = "main"   # change if your repo uses another branch name
INTERVAL = 5      # seconds between checks

print(f"Configured WATCH_DIR={WATCH_DIR}\nBRANCH={BRANCH}\nINTERVAL={INTERVAL}s")

# === HELPERS ===
def snapshot_dir(path):
    """Return mapping of relative file paths to (mtime, size) for files under path. Skips .git."""
    snap = {}
    for root, dirs, files in os.walk(path):
        dirs[:] = [d for d in dirs if d != ".git"]  # skip .git folder
        for f in files:
            fp = os.path.join(root, f)
            try:
                st = os.stat(fp)
                rel = os.path.relpath(fp, path)
                snap[rel] = (st.st_mtime, st.st_size)
            except FileNotFoundError:
                continue
    return snap

def run_cmd(cmd, cwd=None, timeout=300):
    """Run shell command and return (returncode, stdout, stderr)."""
    try:
        proc = subprocess.run(cmd, cwd=cwd, shell=True, capture_output=True, text=True, timeout=timeout)
        return proc.returncode, proc.stdout.strip(), proc.stderr.strip()
    except subprocess.TimeoutExpired:
        return 1, "", "timeout"

def ensure_git_repo(path):
    if not os.path.isdir(os.path.join(path, ".git")):
        raise RuntimeError(f"Not a git repository: {path}")

def git_sync_once(path, branch="main"):
    """Stage all, commit if there are changes, and push. Returns (success, message)."""
    rc, out, err = run_cmd("git add -A", cwd=path)
    if rc != 0:
        return False, f"git add failed: {err or out}"
    rc, out, err = run_cmd("git status --porcelain", cwd=path)
    if rc != 0:
        return False, f"git status failed: {err or out}"
    if out.strip() == "":
        return True, "No changes to commit."
    msg = f"Auto sync: {datetime.utcnow().isoformat()}Z"
    rc, out, err = run_cmd(f'git commit -m "{msg}"', cwd=path)
    if rc != 0:
        return False, f"git commit failed: {err or out}"
    rc, out, err = run_cmd(f"git push origin {branch}", cwd=path)
    if rc != 0:
        return False, f"git push failed: {err or out}"
    return True, "Pushed successfully."

def watch_and_sync(path, branch="main", interval=5):
    """Watch the directory and auto sync on changes. Interrupt the kernel to stop."""
    ensure_git_repo(path)
    print(f"Watching: {path}")
    prev = snapshot_dir(path)
    try:
        while True:
            time.sleep(interval)
            cur = snapshot_dir(path)
            if cur != prev:
                print(f"Change detected at {datetime.now().isoformat()}")
                success, message = git_sync_once(path, branch=branch)
                print("->", message)
                prev = cur
    except KeyboardInterrupt:
        print("\nWatcher stopped by user (KeyboardInterrupt)")

# === START WATCHER ===
print("\nNotebook ready. To start watching, run this line below 👇")
print("watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)")


Configured WATCH_DIR=C:\Users\lokes\Documents\Projects\statistics-notebooks
BRANCH=main
INTERVAL=5s

Notebook ready. To start watching, run this line below 👇
watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)


In [ ]:
watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)

Watching: C:\Users\lokes\Documents\Projects\statistics-notebooks
Change detected at 2025-11-19T13:23:13.998924
-> Pushed successfully.
